# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234346817972                   -0.50    8.0         
  2   -7.249841381625       -1.81       -1.39    1.0   5.41ms
  3   -7.251126637046       -2.89       -1.92    2.0   6.21ms
  4   -7.251158462868       -4.50       -2.02    2.0   6.53ms
  5   -7.251301797305       -3.84       -2.37    1.0   5.40ms
  6   -7.251336915934       -4.45       -2.96    1.0   5.39ms
  7   -7.251338637207       -5.76       -3.54    1.0   5.47ms
  8   -7.251338772568       -6.87       -3.81    2.0   6.59ms
  9   -7.251338796570       -7.62       -4.43    1.0   5.58ms
 10   -7.251338798493       -8.72       -4.87    2.0   6.74ms
 11   -7.251338798682       -9.72       -5.47    1.0   5.75ms
 12   -7.251338798703      -10.70       -5.93    2.0   6.95ms
 13   -7.251338798704      -11.80       -6.23    2.0   7.11ms
 14   -7.251338798704      -12.61       -6.61    1.0   5.87ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.06153506915797236
[ Info: Arnoldi iteration step 2: normres = 0.7221022335004316
[ Info: Arnoldi iteration step 3: normres = 0.6902375081545293
[ Info: Arnoldi iteration step 4: normres = 0.30233929520735714
[ Info: Arnoldi iteration step 5: normres = 0.39144051205239644
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.89e-02, 7.43e-02, 2.51e-01, 2.89e-01, 2.92e-02)
[ Info: Arnoldi iteration step 6: normres = 0.35914049507376167
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (6.08e-03, 1.84e-01, 2.86e-01, 7.87e-02, 7.89e-02)
[ Info: Arnoldi iteration step 7: normres = 0.0655336043294616
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.96e-04, 1.38e-02, 1.19e-02, 1.96e-02, 5.17e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10351731500696151
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (8.70e